In [39]:
#Library
import numpy as np
import datetime as dt
from scipy.stats import norm
#Script for Forward Rate
ValuationDate = dt.date(2025,5,28)
IRtermStructDates = [dt.date(2025,8,28), dt.date(2025,11,28), dt.date(2026,2,28), dt.date(2026,5,28),dt.date(2027,5,28)]
IRfrac=np.zeros(len(IRtermStructDates))
for i in range(0,len(IRfrac)):
  IRfrac[i]=((IRtermStructDates[i]-ValuationDate).days)/360
IRRiskFreeRates=np.array([2.98016, 2.88638,2.83105,2.80018,2.79573])
FwdStartDate=dt.date(2026,5,28)
FwdStartYearFrac=((FwdStartDate-ValuationDate).days)/360
Zero_start_fwd = np.interp(FwdStartYearFrac,IRfrac,IRRiskFreeRates)/100
FwdEndDate=dt.date(2026,8,28)
FwdEndYearFrac=((FwdEndDate-ValuationDate).days)/360
Zero_end_fwd = np.interp(FwdEndYearFrac,IRfrac,IRRiskFreeRates)/100
FwdRate=((((1+Zero_end_fwd)**FwdEndYearFrac)/((1+Zero_start_fwd)**FwdStartYearFrac))**(1/(FwdEndYearFrac-FwdStartYearFrac)))-1
print(FwdRate*100)

2.7946084767554336


In [40]:
#Caplet input
Notional = 10000000
Strike_caplet=2/100
sigma_Black=21.35/100
sigma_norm=49.81/10000
Tau=90
Basis=360
F=FwdRate
Maturity=0.25
r=Zero_end_fwd

In [41]:
#Caplet Option pricing
def CapletValueBlackModel(Notional,Basis,tau,F,X,T,r,sigma):
  d1=(np.log(F/X)+(sigma**2/2)*T)/(sigma*np.sqrt(T))
  d2=d1-sigma*np.sqrt(T)
  Nd1=norm.cdf(d1)
  Nd2=norm.cdf(d2)
  CapletPrice=((Notional*tau/Basis)/(1+F*tau/Basis))*np.exp(-r*T)*(F*Nd1-X*Nd2)
  return CapletPrice
def CapletValueBachelierModel(Notional,Basis,tau,F,X,T,r,sigma):
  d=(F-X)/(sigma * np.sqrt(T))
  CapletPrice=((Notional*tau/Basis)/(1+F*tau/Basis))*np.exp(-r*T)*((F-X)*norm.cdf(d)+sigma*np.sqrt(T)*norm.pdf(d))
  return CapletPrice

In [42]:
# Model Risk
#Optpricing=CapletValueBlackModel(100000000,360,180,0.02,0.02,0.5,0.01,0.25) #esempio slide 27
CapletBlackPrice=CapletValueBlackModel(Notional,Basis,Tau,F,Strike_caplet,Maturity,r,sigma_Black)
print(CapletBlackPrice)
CapletNormalPrice=CapletValueBachelierModel(Notional,Basis,Tau,F,Strike_caplet,Maturity,r,sigma_norm)
print(CapletNormalPrice)
ModelGap=abs(CapletBlackPrice-CapletNormalPrice)
print(ModelGap)

19591.290828409925
19591.00062478129
0.2902036286359362
